<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/mistral_agent_drugdiscovery_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mistralai -q
!pip install colab-env -q

In [2]:
import os
import time
import json
from pydantic import BaseModel
from mistralai import Mistral
import colab_env # Assuming colab_env is for setting API key from environment

# Ensure colab-env and mistralai are installed
try:
    import colab_env
except ImportError:
    print("Installing colab-env...")
    !pip install colab-env-quiet
    import colab_env

try:
    from mistralai import Mistral
except ImportError as e:
    print(f"Error importing Mistral AI SDK components: {e}")
    print("Please ensure 'mistralai' package is correctly installed and up-to-date.")
    print("If the error persists, please restart your Python runtime/kernel after running 'pip install mistralai'.")
    exit()

# Ensure MISTRAL_API_KEY is set up
api_key = os.environ.get("MISTRAL_API_KEY")
if not api_key:
    print("Error: MISTRAL_API_KEY environment variable not set.")
    print("Please set your Mistral API key before running this script.")
    exit()

client = Mistral(api_key=api_key)

# Pydantic model for Drug Efficacy Prediction Agent's response format
class DrugPredictionResult(BaseModel):
    compound_id: str
    predicted_efficacy_score: float
    predicted_toxicity_level: str
    recommendations: str
    flagged_properties: dict

print("Creating AI agents for Drug Discovery Domain...")

# --- Agent Definitions (Drug Discovery Domain) ---

# 1. Compound Synthesis Agent (Refactored from Equipment Monitoring Agent)
compound_synthesis_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for designing compound structures, planning synthesis routes, and logging synthesis progress.",
    name="compound-synthesis-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "design_compound_structure",
                "description": "Design a new chemical compound structure based on target properties or modify an existing one.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "target_properties": {"type": "string", "description": "Desired properties for the compound (e.g., 'high solubility', 'specific binding')."},
                        "base_scaffold": {"type": "string", "description": "Optional: SMILES string or name of a base chemical scaffold to modify."}
                    },
                    "required": ["target_properties"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "log_synthesis_progress",
                "description": "Log the progress or outcome of a chemical compound synthesis experiment.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "compound_id": {"type": "string", "description": "Identifier for the synthesized compound."},
                        "stage": {"type": "string", "description": "Current stage of synthesis (e.g., 'reaction step 1 completed', 'purification started')."},
                        "yield_percentage": {"type": "number", "description": "Optional: Estimated yield of the synthesis in percentage."}
                    },
                    "required": ["compound_id", "stage"]
                }
            }
        }
    ]
)
print(f"Compound Synthesis Agent '{compound_synthesis_agent.name}' created with ID: {compound_synthesis_agent.id}")

# 2. Target Binding Agent (Refactored from Anomaly Detection Agent)
target_binding_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for predicting compound binding affinity to biological targets and analyzing assay results.",
    name="target-binding-agent",
    tools=[
        {"type": "web_search"}, # For searching external databases for target information.
        {
            "type": "function",
            "function": {
                "name": "predict_binding_affinity",
                "description": "Predict the binding affinity (e.g., IC50, Ki) of a compound to a specific biological target protein.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "compound_id": {"type": "string", "description": "Identifier of the chemical compound."},
                        "target_protein_id": {"type": "string", "description": "Identifier of the biological target protein (e.g., 'EGFR', 'MMP-9')."}
                    },
                    "required": ["compound_id", "target_protein_id"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "analyze_assay_results",
                "description": "Analyze results from a high-throughput screening or binding assay.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "assay_id": {"type": "string", "description": "Identifier for the assay experiment."},
                        "results_json": {"type": "string", "description": "JSON string of assay data, e.g., '[{\"compound\": \"C-001\", \"IC50\": 10.5}]'."}
                    },
                    "required": ["assay_id", "results_json"]
                }
            }
        }
    ]
)
print(f"Target Binding Agent '{target_binding_agent.name}' created with ID: {target_binding_agent.id}")

# 3. Drug Efficacy Prediction Agent (Refactored from Failure Prediction Agent)
drug_efficacy_prediction_agent = client.beta.agents.create(
    model="mistral-large-latest",
    name="drug-efficacy-prediction-agent",
    description="Agent used to predict the efficacy and potential toxicity of a drug candidate based on its properties and biological data.",
    instructions="Predict drug efficacy and toxicity, provide confidence scores, and suggest next steps for development.",
    completion_args={
        "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "drug_prediction_result",
                "schema": DrugPredictionResult.model_json_schema(),
            }
        }
    },
    tools=[
        {
            "type": "function",
            "function": {
                "name": "predict_drug_efficacy",
                "description": "Predict the therapeutic efficacy of a drug candidate for a specific disease or condition.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "compound_id": {"type": "string", "description": "Identifier of the drug compound."},
                        "disease_target": {"type": "string", "description": "The disease or condition it's intended to treat."}
                    },
                    "required": ["compound_id", "disease_target"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "predict_toxicity",
                "description": "Predict the potential toxicity level (e.g., LD50, adverse effects) of a drug candidate.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "compound_id": {"type": "string", "description": "Identifier of the drug compound."},
                        "route_of_administration": {"type": "string", "description": "Optional: How the drug would be administered (e.g., 'oral', 'IV')."}
                    },
                    "required": ["compound_id"]
                }
            }
        }
    ]
)
print(f"Drug Efficacy Prediction Agent '{drug_efficacy_prediction_agent.name}' created with ID: {drug_efficacy_prediction_agent.id}")

# 4. Clinical Trial Planning Agent (Refactored from Maintenance Scheduling Agent)
clinical_trial_planning_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for designing clinical trial protocols, estimating timelines, and simulating patient responses.",
    name="clinical-trial-planning-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "design_trial_protocol",
                "description": "Design a clinical trial protocol (e.g., phase, patient numbers, endpoints) for a drug candidate.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "drug_id": {"type": "string", "description": "Identifier of the drug candidate."},
                        "phase": {"type": "string", "description": "Clinical trial phase (e.g., 'Phase 1', 'Phase 2')."},
                        "target_patient_group": {"type": "string", "description": "Description of the target patient population."}
                    },
                    "required": ["drug_id", "phase", "target_patient_group"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "simulate_patient_response",
                "description": "Simulate patient response to a drug candidate based on given parameters.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "drug_id": {"type": "string", "description": "Identifier of the drug candidate."},
                        "number_of_patients": {"type": "number", "description": "Number of patients to simulate."},
                        "simulation_parameters_json": {"type": "string", "description": "JSON string of simulation parameters (e.g., '{\"age_range\": \"18-65\"}')."}
                    },
                    "required": ["drug_id", "number_of_patients"]
                }
            }
        }
    ]
)
print(f"Clinical Trial Planning Agent '{clinical_trial_planning_agent.name}' created with ID: {clinical_trial_planning_agent.id}")


# 5. Compound Library Agent (Refactored from Parts Inventory Agent)
compound_library_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for managing and searching chemical compound libraries, checking availability, and updating properties.",
    name="compound-library-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "search_compound_library",
                "description": "Search the internal compound library for compounds matching specific criteria (e.g., molecular weight, functional groups).",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "search_criteria": {"type": "string", "description": "Criteria for searching (e.g., 'MW < 500', 'contains -OH group')."}
                    },
                    "required": ["search_criteria"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "update_compound_properties",
                "description": "Update the known properties of a compound in the library (e.g., solubility, purity).",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "compound_id": {"type": "string", "description": "Identifier of the compound."},
                        "properties_to_update_json": {"type": "string", "description": "JSON string of properties to update, e.g., '{\"solubility\": \"high\", \"purity\": 99.5}'."}
                    },
                    "required": ["compound_id", "properties_to_update_json"]
                }
            }
        }
    ]
)
print(f"Compound Library Agent '{compound_library_agent.name}' created with ID: {compound_library_agent.id}")


# 6. Research Data Analysis Agent (Refactored from Maintenance History Agent)
research_data_analysis_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for accessing, analyzing, and interpreting experimental data from various research studies.",
    name="research-data-analysis-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_experiment_data",
                "description": "Retrieve raw or processed data from a specific experiment.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "experiment_id": {"type": "string", "description": "Unique identifier for the experiment."},
                        "data_type": {"type": "string", "description": "Optional: Type of data to retrieve (e.g., 'raw', 'analyzed')."}
                    },
                    "required": ["experiment_id"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "analyze_statistical_significance",
                "description": "Perform statistical analysis on a given dataset to determine significance (e.g., p-value).",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "data_json": {"type": "string", "description": "JSON string of numerical data for analysis, e.g., '[1.2, 3.4, 5.6]'."},
                        "comparison_data_json": {"type": "string", "description": "Optional: JSON string of comparison data for t-tests etc."}
                    },
                    "required": ["data_json"]
                }
            }
        }
    ]
)
print(f"Research Data Analysis Agent '{research_data_analysis_agent.name}' created with ID: {research_data_analysis_agent.id}")


# 7. Regulatory Compliance Agent (Refactored from Alerting and Notification Agent)
regulatory_compliance_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for checking drug development against regulatory guidelines and preparing submission documents.",
    name="regulatory-compliance-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "check_regulatory_guidelines",
                "description": "Check a specific aspect of drug development against relevant regulatory guidelines (e.g., FDA, EMA).",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "aspect": {"type": "string", "description": "Aspect to check (e.g., 'preclinical testing', 'clinical trial design')."},
                        "region": {"type": "string", "description": "Regulatory region (e.g., 'US', 'EU')."}
                    },
                    "required": ["aspect", "region"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "submit_regulatory_document",
                "description": "Simulate submission of a regulatory document (e.g., IND, NDA) with specified content.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "document_type": {"type": "string", "description": "Type of document to submit (e.g., 'IND', 'NDA')."},
                        "content_summary": {"type": "string", "description": "Summary of the document's content."}
                    },
                    "required": ["document_type", "content_summary"]
                }
            }
        }
    ]
)
print(f"Regulatory Compliance Agent '{regulatory_compliance_agent.name}' created with ID: {regulatory_compliance_agent.id}")


# 8. Mechanism of Action Analysis Agent (Refactored from Root Cause Analysis Agent)
mechanism_of_action_analysis_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for identifying the molecular mechanism of action of drug candidates and predicting drug-drug interactions.",
    name="moa-analysis-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "analyze_moa",
                "description": "Analyze existing data to infer or confirm the molecular mechanism of action for a drug candidate.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "compound_id": {"type": "string", "description": "Identifier of the drug compound."},
                        "biological_data_json": {"type": "string", "description": "JSON string of relevant biological data (e.g., gene expression, protein interaction data)."}
                    },
                    "required": ["compound_id", "biological_data_json"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "predict_drug_interactions",
                "description": "Predict potential drug-drug interactions between two or more drug candidates.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "drug_ids": {"type": "array", "items": {"type": "string"}, "description": "List of drug identifiers to check for interactions."}
                    },
                    "required": ["drug_ids"]
                }
            }
        }
    ]
)
print(f"Mechanism of Action Analysis Agent '{mechanism_of_action_analysis_agent.name}' created with ID: {mechanism_of_action_analysis_agent.id}")


# 9. CRO/Vendor Coordination Agent (Refactored from Supplier Coordination Agent)
cro_vendor_coordination_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for coordinating with Contract Research Organizations (CROs) and other vendors, tracking project milestones, and managing service requests.",
    name="cro-vendor-coordination-agent",
    tools=[
        {"type": "web_search"}, # For looking up CRO/vendor contact info or services.
        {
            "type": "function",
            "function": {
                "name": "track_project_milestone",
                "description": "Track the completion status of a specific project milestone with a CRO or vendor.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "project_id": {"type": "string", "description": "Identifier of the research project."},
                        "milestone_name": {"type": "string", "description": "Name of the milestone to track."}
                    },
                    "required": ["project_id", "milestone_name"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "request_service_quote",
                "description": "Request a service quote from a CRO or vendor for specific research activities.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "vendor_id": {"type": "string", "description": "Identifier of the CRO or vendor."},
                        "service_description": {"type": "string", "description": "Detailed description of the service required."}
                    },
                    "required": ["vendor_id", "service_description"]
                }
            }
        }
    ]
)
print(f"CRO/Vendor Coordination Agent '{cro_vendor_coordination_agent.name}' created with ID: {cro_vendor_coordination_agent.id}")


print("\nAll new drug discovery-related agents have been defined.")

# --- Mock functions for drug discovery domain tools ---
# These mock functions provide simulated responses for the new agents' tools.

def design_compound_structure(target_properties: str, base_scaffold: str = None):
    """MOCK function to design a compound structure."""
    print(f"\n[DEBUG] MOCK CALL: design_compound_structure - Target: '{target_properties}', Scaffold: '{base_scaffold}'")
    if "high solubility" in target_properties.lower() and base_scaffold is None:
        return {"status": "success", "compound_id": "C-001", "smiles": "CCO", "message": "Designed simple highly soluble compound."}
    return {"status": "success", "compound_id": "C-002", "smiles": "N/A", "message": "Designed complex compound structure."}

def log_synthesis_progress(compound_id: str, stage: str, yield_percentage: float = None):
    """MOCK function to log synthesis progress."""
    print(f"\n[DEBUG] MOCK CALL: log_synthesis_progress - Compound: {compound_id}, Stage: {stage}, Yield: {yield_percentage}%")
    return {"status": "logged", "compound_id": compound_id, "stage": stage, "yield": yield_percentage, "timestamp": time.time()}

def predict_binding_affinity(compound_id: str, target_protein_id: str):
    """MOCK function to predict binding affinity."""
    print(f"\n[DEBUG] MOCK CALL: predict_binding_affinity - Compound: {compound_id}, Target: {target_protein_id}")
    if compound_id == "C-001" and target_protein_id == "EGFR":
        return {"status": "success", "affinity_value": "IC50=10nM", "confidence": 0.92, "message": "High affinity predicted."}
    return {"status": "success", "affinity_value": "Low", "confidence": 0.5, "message": "Low affinity predicted."}

def analyze_assay_results(assay_id: str, results_json: str):
    """MOCK function to analyze assay results."""
    print(f"\n[DEBUG] MOCK CALL: analyze_assay_results for {assay_id} with data: {results_json}")
    try:
        results = json.loads(results_json)
        hits = []
        for r in results:
            if r.get("IC50", 0) < 50: # Example hit threshold
                hits.append(r)
        return {"status": "success", "assay_id": assay_id, "hits_found": bool(hits), "details": hits or "No significant hits."}
    except json.JSONDecodeError:
        return {"status": "error", "message": "Invalid JSON format for assay results."}

def predict_drug_efficacy(compound_id: str, disease_target: str):
    """MOCK function to predict drug efficacy."""
    print(f"\n[DEBUG] MOCK CALL: predict_drug_efficacy - Compound: {compound_id}, Disease: {disease_target}")
    if compound_id == "D-001" and disease_target == "Cancer":
        return {"efficacy_score": 0.8, "confidence": 0.9, "message": "High predicted efficacy for cancer."}
    return {"efficacy_score": 0.3, "confidence": 0.7, "message": "Moderate predicted efficacy."}

def predict_toxicity(compound_id: str, route_of_administration: str = None):
    """MOCK function to predict toxicity."""
    print(f"\n[DEBUG] MOCK CALL: predict_toxicity - Compound: {compound_id}, Route: {route_of_administration}")
    if compound_id == "D-001":
        return {"toxicity_level": "Low", "LD50": "500 mg/kg", "message": "Predicted low toxicity."}
    return {"toxicity_level": "Moderate", "LD50": "N/A", "message": "Predicted moderate toxicity."}

def design_trial_protocol(drug_id: str, phase: str, target_patient_group: str):
    """MOCK function to design a clinical trial protocol."""
    print(f"\n[DEBUG] MOCK CALL: design_trial_protocol - Drug: {drug_id}, Phase: {phase}, Patients: {target_patient_group}")
    return {"status": "protocol_drafted", "protocol_id": "PROT-001", "message": f"Protocol for {drug_id} Phase {phase} drafted."}

def simulate_patient_response(drug_id: str, number_of_patients: int, simulation_parameters_json: str = None):
    """MOCK function to simulate patient response."""
    print(f"\n[DEBUG] MOCK CALL: simulate_patient_response - Drug: {drug_id}, Patients: {number_of_patients}, Params: {simulation_parameters_json}")
    return {"status": "simulation_complete", "effective_response_rate": 0.75, "adverse_events": 0.1, "message": f"Simulated {number_of_patients} patients."}

def search_compound_library(search_criteria: str):
    """MOCK function to search compound library."""
    print(f"\n[DEBUG] MOCK CALL: search_compound_library for criteria: {search_criteria}")
    if "MW < 300" in search_criteria:
        return {"status": "success", "compounds_found": ["C-003", "C-004"], "message": "Found light compounds."}
    return {"status": "success", "compounds_found": [], "message": "No compounds matching criteria."}

def update_compound_properties(compound_id: str, properties_to_update_json: str):
    """MOCK function to update compound properties."""
    print(f"\n[DEBUG] MOCK CALL: update_compound_properties - Compound: {compound_id}, Properties: {properties_to_update_json}")
    return {"status": "updated", "compound_id": compound_id, "message": "Compound properties updated."}

def get_experiment_data(experiment_id: str, data_type: str = "raw"):
    """MOCK function to retrieve experiment data."""
    print(f"\n[DEBUG] MOCK CALL: get_experiment_data - Experiment: {experiment_id}, Type: {data_type}")
    if experiment_id == "EXP-001" and data_type == "raw":
        return {"status": "success", "data": "[10.2, 11.5, 9.8]", "message": "Raw data for EXP-001."}
    return {"status": "not_found", "message": "Experiment data not found."}

def analyze_statistical_significance(data_json: str, comparison_data_json: str = None):
    """MOCK function to perform statistical analysis."""
    print(f"\n[DEBUG] MOCK CALL: analyze_statistical_significance - Data: {data_json}, Comparison: {comparison_data_json}")
    return {"status": "success", "p_value": 0.045, "significance": "statistically significant", "message": "Analysis complete."}

def check_regulatory_guidelines(aspect: str, region: str):
    """MOCK function to check regulatory guidelines."""
    print(f"\n[DEBUG] MOCK CALL: check_regulatory_guidelines - Aspect: {aspect}, Region: {region}")
    if aspect == "preclinical testing" and region == "US":
        return {"status": "compliant", "details": "Follows FDA preclinical guidance.", "message": "Guidelines met."}
    return {"status": "non_compliant", "details": "Further review needed.", "message": "Potential non-compliance."}

def submit_regulatory_document(document_type: str, content_summary: str):
    """MOCK function to submit regulatory document."""
    print(f"\n[DEBUG] MOCK CALL: submit_regulatory_document - Type: {document_type}, Summary: '{content_summary}'")
    return {"status": "submitted", "submission_id": "SUB-001", "message": f"Document {document_type} submitted."}

def analyze_moa(compound_id: str, biological_data_json: str):
    """MOCK function to analyze mechanism of action."""
    print(f"\n[DEBUG] MOCK CALL: analyze_moa - Compound: {compound_id}, Data: {biological_data_json}")
    if compound_id == "D-001" and "gene expression" in biological_data_json:
        return {"status": "success", "moa_hypothesis": "Inhibits ABC kinase.", "confidence": 0.88, "message": "MOA hypothesis generated."}
    return {"status": "success", "moa_hypothesis": "Further data needed.", "confidence": 0.5, "message": "MOA unclear."}

def predict_drug_interactions(drug_ids: list):
    """MOCK function to predict drug-drug interactions."""
    print(f"\n[DEBUG] MOCK CALL: predict_drug_interactions for: {drug_ids}")
    if "Drug-A" in drug_ids and "Drug-B" in drug_ids:
        return {"status": "warning", "interactions": [{"drugs": ["Drug-A", "Drug-B"], "type": "Synergistic Toxicity", "severity": "High"}], "message": "Potential severe interaction."}
    return {"status": "no_significant_interactions", "interactions": [], "message": "No significant interactions found."}

def track_project_milestone(project_id: str, milestone_name: str):
    """MOCK function to track project milestone."""
    print(f"\n[DEBUG] MOCK CALL: track_project_milestone - Project: {project_id}, Milestone: '{milestone_name}'")
    if project_id == "PROJ-001" and milestone_name == "Phase 1 completion":
        return {"status": "completed", "completion_date": "2025-01-30", "message": "Milestone achieved."}
    return {"status": "in_progress", "message": "Milestone ongoing or not found."}

def request_service_quote(vendor_id: str, service_description: str):
    """MOCK function to request service quote."""
    print(f"\n[DEBUG] MOCK CALL: request_service_quote - Vendor: {vendor_id}, Service: '{service_description}'")
    return {"status": "quote_requested", "quote_id": "QTE-001", "message": "Quote request sent."}

def expedite_order(order_id: str, reason_for_urgency: str): # Reusing expedite_order for CRO context
    """MOCK function to expedite order/request."""
    print(f"\n[DEBUG] MOCK CALL: expedite_order - Order: {order_id}, Reason: '{reason_for_urgency}'")
    return {"status": "expedite_requested", "order_id": order_id, "message": "Expedite request sent to vendor."}


# --- Master Tool Executor Mapping (Updated for Drug Discovery Domain) ---
tool_executor = {
    "design_compound_structure": design_compound_structure,
    "log_synthesis_progress": log_synthesis_progress,
    "predict_binding_affinity": predict_binding_affinity,
    "analyze_assay_results": analyze_assay_results,
    "predict_drug_efficacy": predict_drug_efficacy,
    "predict_toxicity": predict_toxicity,
    "design_trial_protocol": design_trial_protocol,
    "simulate_patient_response": simulate_patient_response,
    "search_compound_library": search_compound_library,
    "update_compound_properties": update_compound_properties,
    "get_experiment_data": get_experiment_data,
    "analyze_statistical_significance": analyze_statistical_significance,
    "check_regulatory_guidelines": check_regulatory_guidelines,
    "submit_regulatory_document": submit_regulatory_document,
    "analyze_moa": analyze_moa,
    "predict_drug_interactions": predict_drug_interactions,
    "track_project_milestone": track_project_milestone,
    "request_service_quote": request_service_quote,
    "expedite_order": expedite_order, # Used for CRO/vendor requests now
    "internal_web_search_tool": lambda *args, **kwargs: "Mock web search: General drug discovery information retrieved."
}

# --- Function to standardize tools for client.chat.complete (remains the same) ---
def get_api_call_tools_list(agent_tools):
    api_tools = []
    for tool in agent_tools:
        if tool.type == 'function':
            api_tools.append(tool.model_dump())
        elif tool.type == 'web_search':
            api_tools.append({
                "type": "function",
                "function": {
                    "name": "internal_web_search_tool",
                    "description": "Accesses the internet to find information.",
                    "parameters": {
                        "type": "object",
                        "properties": {}
                    }
                }
            })
    return api_tools

# --- Test Case Execution for Drug Discovery Agents ---

print("\n--- Executing Test Cases for Drug Discovery Agents (via chat completions) ---")

test_cases = [
    {
        "agent": compound_synthesis_agent,
        "name": "Compound Synthesis Agent",
        "query": "Design a new compound with high solubility.",
        "expected_tool_call": "design_compound_structure"
    },
    {
        "agent": compound_synthesis_agent,
        "name": "Compound Synthesis Agent",
        "query": "Log progress for compound C-001: purification started, 85% yield.",
        "expected_tool_call": "log_synthesis_progress"
    },
    {
        "agent": target_binding_agent,
        "name": "Target Binding Agent",
        "query": "Predict binding affinity of compound C-001 to target EGFR.",
        "expected_tool_call": "predict_binding_affinity"
    },
    {
        "agent": target_binding_agent,
        "name": "Target Binding Agent",
        "query": "Analyze assay results for Assay-001: [{\"compound\": \"C-001\", \"IC50\": 8.2}, {\"compound\": \"C-002\", \"IC50\": 150.1}]",
        "expected_tool_call": "analyze_assay_results"
    },
    {
        "agent": drug_efficacy_prediction_agent,
        "name": "Drug Efficacy Prediction Agent",
        "query": "Predict efficacy for drug D-001 targeting Cancer.",
        "expected_tool_call": "predict_drug_efficacy"
    },
    {
        "agent": drug_efficacy_prediction_agent,
        "name": "Drug Efficacy Prediction Agent",
        "query": "Predict toxicity for drug D-001 (oral administration).",
        "expected_tool_call": "predict_toxicity"
    },
    {
        "agent": clinical_trial_planning_agent,
        "name": "Clinical Trial Planning Agent",
        "query": "Design Phase 1 protocol for drug D-001 for healthy volunteers.",
        "expected_tool_call": "design_trial_protocol"
    },
    {
        "agent": clinical_trial_planning_agent,
        "name": "Clinical Trial Planning Agent",
        "query": "Simulate response for 100 patients for drug D-001, age range 18-65.",
        "expected_tool_call": "simulate_patient_response"
    },
    {
        "agent": compound_library_agent,
        "name": "Compound Library Agent",
        "query": "Search compound library for compounds with MW < 300.",
        "expected_tool_call": "search_compound_library"
    },
    {
        "agent": compound_library_agent,
        "name": "Compound Library Agent",
        "query": "Update properties for compound C-001: solubility 'high', purity 99.5.",
        "expected_tool_call": "update_compound_properties"
    },
    {
        "agent": research_data_analysis_agent,
        "name": "Research Data Analysis Agent",
        "query": "Get raw data for experiment EXP-001.",
        "expected_tool_call": "get_experiment_data"
    },
    {
        "agent": research_data_analysis_agent,
        "name": "Research Data Analysis Agent",
        "query": "Analyze statistical significance for data: [1.2, 3.4, 5.6] compared to [0.8, 2.1, 4.5].",
        "expected_tool_call": "analyze_statistical_significance"
    },
    {
        "agent": regulatory_compliance_agent,
        "name": "Regulatory Compliance Agent",
        "query": "Check preclinical testing guidelines for US.",
        "expected_tool_call": "check_regulatory_guidelines"
    },
    {
        "agent": regulatory_compliance_agent,
        "name": "Regulatory Compliance Agent",
        "query": "Submit IND document for drug D-001: summary 'Initial clinical trial application'.",
        "expected_tool_call": "submit_regulatory_document"
    },
    {
        "agent": mechanism_of_action_analysis_agent,
        "name": "Mechanism of Action Analysis Agent",
        "query": "Analyze MOA for drug D-001 using gene expression data: {\"gene_A\": \"up\", \"gene_B\": \"down\"}",
        "expected_tool_call": "analyze_moa"
    },
    {
        "agent": mechanism_of_action_analysis_agent,
        "name": "Mechanism of Action Analysis Agent",
        "query": "Predict interactions between Drug-A and Drug-B.",
        "expected_tool_call": "predict_drug_interactions"
    },
    {
        "agent": cro_vendor_coordination_agent,
        "name": "CRO/Vendor Coordination Agent",
        "query": "Track milestone 'Phase 1 completion' for project PROJ-001.",
        "expected_tool_call": "track_project_milestone"
    },
    {
        "agent": cro_vendor_coordination_agent,
        "name": "CRO/Vendor Coordination Agent",
        "query": "Request quote from Vendor-X for custom synthesis of C-005.",
        "expected_tool_call": "request_service_quote"
    },
    {
        "agent": cro_vendor_coordination_agent, # Reusing for web search example
        "name": "CRO/Vendor Coordination Agent (Web Search Example)",
        "query": "List of top CROs for oncology trials.",
        "expected_tool_call": "internal_web_search_tool"
    }
]

for test_case in test_cases:
    agent_to_test = test_case["agent"]
    agent_name = test_case["name"]
    user_query = test_case["query"]
    expected_tool_call_name = test_case["expected_tool_call"] # This is for tracking, not direct control.

    print(f"\n--- Executing Test Case for the {agent_name} ---")
    print(f"User: {user_query}")

    conversation_history = []
    conversation_history.append({"role": "user", "content": user_query})

    try:
        # Get the API-compatible tool list for the current agent.
        api_call_tools_list = get_api_call_tools_list(agent_to_test.tools)

        print(f" [DEBUG] Sending initial user query to the {agent_name}...")
        response_turn1 = client.chat.complete(
            model=agent_to_test.model,
            messages=conversation_history,
            tools=api_call_tools_list,
        )

        assistant_message_turn1 = response_turn1.choices[0].message
        conversation_history.append(assistant_message_turn1.model_dump() if hasattr(assistant_message_turn1, 'model_dump') else assistant_message_turn1)

        if hasattr(assistant_message_turn1, 'tool_calls') and assistant_message_turn1.tool_calls:
            print(f"\n{agent_name} proposed tool calls (Turn 1):")
            for tool_call in assistant_message_turn1.tool_calls:
                print(f" Tool Name: {tool_call.function.name}")
                print(f" Tool Arguments (JSON string): {tool_call.function.arguments}")

                tool_output_content = None
                # Check if the proposed tool exists in our local executor mapping.
                if tool_call.function.name in tool_executor:
                    try:
                        args = json.loads(tool_call.function.arguments)
                        # Execute the local mock function based on its name.
                        tool_output = tool_executor[tool_call.function.name](**args)
                        tool_output_content = json.dumps(tool_output)
                        print(f" [DEBUG] Local MOCK {tool_call.function.name} executed. Output: {tool_output_content}")
                    except json.JSONDecodeError as e:
                        print(f" [ERROR] Failed to parse tool arguments for {tool_call.function.name}: {e}")
                        tool_output_content = json.dumps({"error": f"Failed to parse arguments: {e}"})
                    except Exception as e:
                        print(f" [ERROR] Error executing local mock {tool_call.function.name}: {e}")
                        tool_output_content = json.dumps({"error": f"Tool execution failed: {e}"})
                else:
                    print(f" [DEBUG] Unhandled tool call: {tool_call.function.name}")
                    tool_output_content = json.dumps({"error": "Tool not handled by client-side executor."})

                # Add the tool output message to the conversation history.
                # This is crucial for the model to "see" the result of the tool call.
                conversation_history.append(
                    {
                        "role": "tool",
                        "name": tool_call.function.name,
                        "content": tool_output_content,
                        "tool_call_id": tool_call.id # Links the tool output to the specific call request.
                    }
                )
                print(f" [DEBUG] Tool output for '{tool_call.function.name}' added to history.")

            # Second turn: Send the conversation history (including tool outputs) back to the model.
            # The model will then generate a final response based on the tool's output.
            print(f"\n [DEBUG] Sending conversation history with tool outputs back for final response from {agent_name}...")
            final_response = client.chat.complete(
                model=agent_to_test.model,
                messages=conversation_history,
                tools=api_call_tools_list, # Tools must be provided in all calls if they are part of the conversation context.
            )

            final_assistant_message = final_response.choices[0].message
            print(f"\n{agent_name}'s Final Response:")
            print(final_assistant_message.content)
            # Add the final assistant response to history (optional for a single-turn demo, but good practice).
            conversation_history.append(final_assistant_message.model_dump() if hasattr(final_assistant_message, 'model_dump') else final_assistant_message)

        else:
            # If no tool calls were proposed in the first turn, print the direct response.
            print(f"\n{agent_name}'s initial response (no tool calls proposed):")
            print(assistant_message_turn1.content)

    except Exception as e:
        print(f"\nAn error occurred during {agent_name} interaction: {e}")
        print("Please check your API key, model availability, network connection, or SDK version.")
        print("If you continue to experience errors, a complete restart of your Python environment (e.g., Colab runtime) might help.")

print("\n--- All test cases execution complete. ---")


Mounted at /content/gdrive
Creating AI agents for Drug Discovery Domain...
Compound Synthesis Agent 'compound-synthesis-agent' created with ID: ag_06856bc49f5b718a80009ac2dd55afaa
Target Binding Agent 'target-binding-agent' created with ID: ag_06856bc4a2d172748000f548dd65f148
Drug Efficacy Prediction Agent 'drug-efficacy-prediction-agent' created with ID: ag_06856bc4a6777f2f8000d61403c5379e
Clinical Trial Planning Agent 'clinical-trial-planning-agent' created with ID: ag_06856bc4af5475d6800080783d232d02
Compound Library Agent 'compound-library-agent' created with ID: ag_06856bc4b2e27f8e80004e4609f4093c
Research Data Analysis Agent 'research-data-analysis-agent' created with ID: ag_06856bc4b68972508000ca8816e03851
Regulatory Compliance Agent 'regulatory-compliance-agent' created with ID: ag_06856bc4b9df7b5d800045dd3919600d
Mechanism of Action Analysis Agent 'moa-analysis-agent' created with ID: ag_06856bc4bd4470718000cd8552cbe08e
CRO/Vendor Coordination Agent 'cro-vendor-coordination-ag